# Title: Vehicle detection

## 1. Analysis

In [ ]:
from __future__ import print_function

Future is basically the missing compatibilty layer betweeb python 2 and python 3. It allows use to use python 3.x codebase with minimal overhead. 

In [ ]:
from skimage.feature import hog

In [ ]:
import matplotlib.image as mpimg

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
import cv2

In [ ]:
import glob

In [ ]:
import time

This module provides various time-related functions. For related functionality, see also the datetime and calendar modules.

In [2]:
from sklearn.svm import LinearSVC

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data.

In [3]:
from sklearn.preprocessing import StandardScaler

Standardize a dataset along any axis.
Center to the mean and component wise scale to unit variance.

In [4]:
from sklearn.model_selection import train_test_split

Model selection and evaluation using tools, such as model_selection.GridSearchCV and model_selection.cross_val_score, take a scoring parameter that controls what metric they apply to the estimators evaluated.

In [5]:
import pickle

The pickle module implements a fundamental, but powerful algorithm for serializing and de-serializing a Python object structure. “Pickling” is the process whereby a Python object hierarchy is converted into a byte stream, and “unpickling” is the inverse operation, whereby a byte stream is converted back into an object hierarchy.

In [ ]:
#image = mpimg.imread('test_images/test1.jpg')
#return value of imread is numpy.array
#define a function to compute a colour histogram
#We need to change bin_range if reading .png file with mpimg
def color_hist(image, nbins=32, bins_range=(0,256)):
    #compute the histogram for RGB channels separately
    rhist = np.histogram(image[:,:,0], bins = 32, range = (0,256))
    ghist = np.histogram(image[:,:,1], bins = 32, range = (0,256))
    bhist = np.histogram(image[:,:,2], bins = 32, range = (0,256))

    #Generating bin centers
    bin_edges = rhist[1]
    bin_centers = (bin_edges[1:] + bin_edges[0:len(bin_edges)-1])/2

    #Now we concatenate the histograms for R, G, B into a single feature vector
    hist_features = np.concatenate((rhist[0], ghist[0], bhist[0]))
    #return rhist, ghist, bhist, bin_centers, hist_features
    #We will only return the list of r,g,b channels
    #print("Type of hist features:",type(hist_features))
    #print("Len of hist features: ", len(hist_features))
    return hist_features

#define a function to compute binned color features
def bin_spatial(img, size=(32, 32)):

    features = cv2.resize(img, size).ravel()
    #ravel() creates a flattened 1-D array
    return features


#define a function to return HOG features and visualization
def get_hog_features(img, orient = 8, pix_per_cell = 8, cell_per_bock = 2, vis = True, feature_vec = True):
    #If visualization is turned on ie vis == True, then return even the visualization. Else just return the features
    if vis == True:
        features, hog_image = hog(img, orientations=orient,
                                  pixels_per_cell=(pix_per_cell,pix_per_cell),
                                  cells_per_block=(cell_per_bock,cell_per_bock),
                                  visualise=vis, transform_sqrt=False,
                                  feature_vector=feature_vec, normalise=None)
        return features, hog_image
    else:
        features = hog(img, orientations=orient,
                       pixels_per_cell = (pix_per_cell,pix_per_cell),
                       cells_per_block=(cell_per_bock,cell_per_bock),
                       visualise=vis, transform_sqrt=False,
                       feature_vector=feature_vec, normalise=None)
        return features

#define a function to extract features from a list of images
#This function will call bin_spatial() and color_hist()
def extract_features(images, color_space = 'RGB', spatial_size = (32,32),
                     hist_bins = 32, orient = 9,
                     pix_per_cell = 8, cell_per_block = 2, hog_channel = 0,
                     spatial_feat = True, hist_feat = True, hog_feat = True,
                     viz = False, viz_only = False, hog_viz_name = "", viz_title = "HOG Visualization"):

    #Create a list to append the feature vectors of all images
    features = []
    for file in images:
        #File to store features of individual images
        file_features = []
        img = mpimg.imread(file)
        #The value of img will be the dimensions width X height given as array. Type is numpy.ndarray
        #print("The value of img on reading from the vector of image names: ", len(img))
        #Apply color conversion if any
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            if color_space == 'LUV':
                feature_image = cv2.cvtCcolor(img, cv2.COLOR_RGB2LUV)
            if color_space == 'HLS':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
            if color_space == 'YUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
            if color_space == 'YCrCb':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else:
            feature_image = np.copy(img)
        #print("The values of feature image on applying a color conversion: ", feature_image)
        #print("The value and type of feature image: ", len(feature_image), type(feature_image))
        #Apply Spatial binning and color conversion
        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, spatial_size)
            file_features.append(spatial_features)
        #print("The value of file_features after applying a spatial bin for 1 image : ",file_features, len(file_features[0]), type(file_features[0]))
        #print("Length of file_features on appending spatial_features is: ", len(file_features))
        #Apply color histogram
        if hist_feat == True:
            hist_features = color_hist(feature_image, nbins=hist_bins)
            #print("The returned value of hist features is :", hist_features)
            #print("Length is: ", len(hist_features))
            file_features.append(hist_features)
        #print("Length of file_features on appending hist_features is: ", len(file_features))
        #print("The value of file_features after applying hist_features is: ", file_features, file_features[1], type(file_features[1]))
        #Call get_hog_features() with vis = False, feature_vec = True
        if hog_feat == True:
            if viz == True:
                if hog_channel == 'ALL':
                    hog_features = []
                    #print("\nFeature image.shape: ", feature_image.shape)
                    for channel in range(feature_image.shape[2]):
                        hog_feature, hog_image = get_hog_features(feature_image[:,:,channel],
                                                                  orient, pix_per_cell, cell_per_block,
                                                                  vis=True, feature_vec=True)
                        hog_features.append(hog_feature)
                    hog_features = np.ravel(hog_features)
                    #print("The hog_features on setting channel as ALL are: ", hog_features, len(hog_features))

                #If HOG channel is 0,1,2 ie R,G,B, then do the following function
                else:
                    hog_features, hog_image = get_hog_features(feature_image[:,:,hog_channel],
                                                               orient, pix_per_cell, cell_per_block,
                                                               vis=True, feature_vec=True)
                    print("The hog_features are: ", len(hog_features), type(hog_features))
                    print("The hog_image: ", len(hog_image), type(hog_image))
                #Plot HOG visualization
                #Show only ther HOG image
                if viz_only == True:
                    fig = plt.figure()
                    plt.imshow(hog_image)
                    plt.title(viz_title)
                    #plt.show()
                    #plt.savefig(hog_viz_name, bbox_inches='tight')
                #Show both the original and HOG image side by side for better comparison
                else:
                    fig = plt.figure()
                    plt.subplot(121)
                    plt.imshow(feature_image)
                    plt.title('Sample image')
                    plt.subplot(122)
                    plt.imshow(hog_image)
                    plt.title('Example HOG visualization')
                    #plt.show()
                    #plt.savefig(hog_viz_name + "_double", bbox_inches='tight')

            #Now, if viz == False
            #Same as the previous step, except that here we are not going to plot the images.
            else:
                if hog_channel == 'ALL':
                    hog_features = []
                    #print("\nFeature image.shape: ", feature_image.shape)
                    for channel in range(feature_image.shape[2]):
                        hog_feature = get_hog_features(feature_image[:,:,channel],
                                                       orient, pix_per_cell, cell_per_block,
                                                       vis=False, feature_vec=True)
                        hog_features.append(hog_feature)
                    hog_features = np.ravel(hog_features)
                    #print("The hog_features on setting channel as ALL are: ", hog_features, len(hog_features))
                else:
                    hog_features = get_hog_features(feature_image[:,:,hog_channel],
                                                    orient, pix_per_cell, cell_per_block,
                                                    vis=False, feature_vec=True)

            #Append the new feature vector to the features list
            file_features.append(hog_features)
            #print("Length of file_featuers on appending hog_features : ", len(file_features))
        features.append(np.concatenate(file_features))
        #print("The length of the global features list, that contain the features of all the images read: ", len(features))
    #Return the list of feature vectors
    return features

cars = glob.glob('vehicles/**/*.png', recursive = True)
not_cars = glob.glob('non-vehicles/**/*.png', recursive = True)

#Reduce the sample size
sample_size = 10
cars = cars[0:sample_size]
not_cars = not_cars[0:sample_size]

#select one car by index randomly
sample_car = [cars[5]]
sample_not_car = [not_cars[5]]
print(sample_car)

#call the funtion on the image/images
car_features = extract_features(sample_car, viz=True, hog_channel='ALL')
not_car_features = extract_features(sample_not_car, viz=True, hog_channel='ALL')

print(len(car_features))
print(len(not_car_features))



## 2. Linear SVM

### 2.1 Set parameters used to call extract features:

In [ ]:

color_space = "RGB"
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 'ALL'
spatial_size = (32,32)
hist_bins = 32
spatial_feat = True
hist_feat = True
hog_feat = True

def color_hist(image, nbins=32, bins_range=(0,256)):
    #compute the histogram for RGB channels separately
    rhist = np.histogram(image[:,:,0], bins = 32, range = (0,256))
    ghist = np.histogram(image[:,:,1], bins = 32, range = (0,256))
    bhist = np.histogram(image[:,:,2], bins = 32, range = (0,256))

    #Generating bin centers
    bin_edges = rhist[1]
    bin_centers = (bin_edges[1:] + bin_edges[0:len(bin_edges)-1])/2

    #Now we concatenate the histograms for R, G, B into a single feature vector
    hist_features = np.concatenate((rhist[0], ghist[0], bhist[0]))
    return hist_features

#define a function to compute binned color features
def bin_spatial(img, size=(32, 32)):

    features = cv2.resize(img, size).ravel()
    #ravel() creates a flattened 1-D array
    return features


#define a function to return HOG features and visualization
def get_hog_features(img, orient = 8, pix_per_cell = 8, cell_per_bock = 2, vis = True, feature_vec = True):
    #If visualization is turned on ie vis == True, then return even the visualization. Else just return the features
    if vis == True:
        features, hog_image = hog(img, orientations=orient,
                                  pixels_per_cell=(pix_per_cell,pix_per_cell),
                                  cells_per_block=(cell_per_bock,cell_per_bock),
                                  visualise=vis, transform_sqrt=False,
                                  feature_vector=feature_vec, normalise=None)
        return features, hog_image
    else:
        features = hog(img, orientations=orient,
                       pixels_per_cell = (pix_per_cell,pix_per_cell),
                       cells_per_block=(cell_per_bock,cell_per_bock),
                       visualise=vis, transform_sqrt=False,
                       feature_vector=feature_vec, normalise=None)
        return features

#define a function to extract features from a list of images
def extract_features(images, color_space = 'RGB', spatial_size = (32,32),
                     hist_bins = 32, orient = 9,
                     pix_per_cell = 8, cell_per_block = 2, hog_channel = 0,
                     spatial_feat = True, hist_feat = True, hog_feat = True,
                     viz = False, viz_only = False, hog_viz_name = "", viz_title = "HOG Visualization"):

    features = []
    for file in images:
        #File to store features of individual images
        file_features = []
        img = mpimg.imread(file)
        #The value of img will be the dimensions width X height given as array. Type is numpy.ndarray

        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            if color_space == 'LUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
            if color_space == 'HLS':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
            if color_space == 'YUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
            if color_space == 'YCrCb':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else:
            feature_image = np.copy(img)

        #Apply Spatial binning and color conversion
        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, spatial_size)
            file_features.append(spatial_features)
        #Apply color histogram
        if hist_feat == True:
            hist_features = color_hist(feature_image, nbins=hist_bins)
            #print("The returned value of hist features is :", hist_features)
            #print("Length is: ", len(hist_features))
            file_features.append(hist_features)
        #Call get_hog_features() with vis = False, feature_vec = True
        if hog_feat == True:
            if viz == True:
                if hog_channel == 'ALL':
                    hog_features = []
                    for channel in range(feature_image.shape[2]):
                        hog_feature, hog_image = get_hog_features(feature_image[:,:,channel],
                                                                  orient, pix_per_cell, cell_per_block,
                                                                  vis=True, feature_vec=True)
                        hog_features.append(hog_feature)
                    hog_features = np.ravel(hog_features)

                else:
                    hog_features, hog_image = get_hog_features(feature_image[:,:,hog_channel],
                                                               orient, pix_per_cell, cell_per_block,
                                                               vis=True, feature_vec=True)
                    print("The hog_features are: ", len(hog_features), type(hog_features))
                    print("The hog_image: ", len(hog_image), type(hog_image))
                #Plot HOG visualization
                #Show only ther HOG image
                if viz_only == True:
                    fig = plt.figure()
                    plt.imshow(hog_image)
                    plt.title(viz_title)
                    #plt.show()
                    #plt.savefig(hog_viz_name, bbox_inches='tight')
                #Show both the original and HOG image side by side for better comparison
                else:
                    fig = plt.figure()
                    plt.subplot(121)
                    plt.imshow(feature_image)
                    plt.title('Sample image')
                    plt.subplot(122)
                    plt.imshow(hog_image)
                    plt.title('Example HOG visualization')
                    #plt.show()
                    #plt.savefig(hog_viz_name + "_double", bbox_inches='tight')

            #Now, if viz == False
            #Same as the previous step, except that here we are not going to plot the images.
            else:
                if hog_channel == 'ALL':
                    hog_features = []
                    for channel in range(feature_image.shape[2]):
                        hog_feature = get_hog_features(feature_image[:,:,channel],
                                                       orient, pix_per_cell, cell_per_block,
                                                       vis=False, feature_vec=True)
                        hog_features.append(hog_feature)
                    hog_features = np.ravel(hog_features)
                else:
                    hog_features = get_hog_features(feature_image[:,:,hog_channel],
                                                    orient, pix_per_cell, cell_per_block,
                                                    vis=False, feature_vec=True)

            #Append the new feature vector to the features list
            file_features.append(hog_features)
            #print("Length of file_featuers on appending hog_features : ", len(file_features))
        features.append(np.concatenate(file_features))
    #Return the list of feature vectors
    return features

cars = glob.glob('vehicles/**/*.png', recursive = True)
not_cars = glob.glob('non-vehicles/**/*.png', recursive = True)

#Reduce the sample size
#sample_size = 5
#cars = cars[0:sample_size]
#not_cars = not_cars[0:sample_size]

#call the funtion on the image/images
car_features = extract_features(cars, color_space=color_space,
                                spatial_size=spatial_size,
                                hist_bins=hist_bins, orient=orient,
                                pix_per_cell=pix_per_cell,
                                cell_per_block=cell_per_block,
                                hog_channel='ALL', spatial_feat=spatial_feat,
                                hist_feat=hist_feat, hog_feat=hog_feat,
                                viz=False)
not_car_features = extract_features(not_cars, color_space=color_space,
                                spatial_size=spatial_size,
                                hist_bins=hist_bins, orient=orient,
                                pix_per_cell=pix_per_cell,
                                cell_per_block=cell_per_block,
                                hog_channel='ALL', spatial_feat=spatial_feat,
                                hist_feat=hist_feat, hog_feat=hog_feat,
                                viz=False)

#Type of x is numpy.ndarray
#it will have the length = 10 cars + 10 not-cars = 20
X = np.vstack((car_features, not_car_features)).astype(np.float64)
#print(X)
#print(type(X))
#print(len(X))

#fit a per-column scaler
X_scaler = StandardScaler().fit(X)
#print(X_scaler)         #Output: StandardScaler(copy=True, with_mean=True, with_std=True)
#print(type(X_scaler))   #Output: <class 'sklearn.preprocessing.data.StandardScaler'>

#Apply the scaler to X
scaled_X = X_scaler.transform(X)
#print(scaled_X)
#print(type(scaled_X))   #numpy.ndarray
#print(len(scaled_X))    # 20 for 10 + 10 examples
#Scaled_X is the array that contains the featuers of individual images and the next list Y, contains the assciated class values
#ie car or not car. Thus, we are assigning the Y array to those many 1's as there are cars and 0's for not cars
#Define the labels vector
Y = np.hstack((np.ones(len(car_features)), np.zeros(len(not_car_features))))
#print(Y)            #[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.   0.  0.]
#print(len(Y))       #len 20
#print(type(Y))      #numpy.ndarray

#Split up data into randomized training and test sets
rand_state = np.random.randint(0,100)
#print(rand_state)
#Randomly assigns a number to rand_state from 0 to 99. This value changes with every new function call
x_train, x_test, y_train, y_test = train_test_split(scaled_X, Y, test_size=0.2, random_state=rand_state)

#print(x_train)
#print(len(x_train))
#print(x_test)
#print(len(x_test))

#print(y_train)
#print(len(y_train))
#print(y_test)
#print(len(y_test))

#print('Feature vector length : ', len(x_train[0]))

#Use a linear SVC
svc = LinearSVC()

#Check the training time for SVC
t1 = time.time()
svc.fit(x_train, y_train)
t2 = time.time()
print(round(t2-t1, 2), ' seconds were required for training the SVC')

#Check the score of the SVC
#print("Total number of car samples: ", len(cars))
#print("Total number of not_car samples: ", len(not_cars))
print('Test accuracy for SVC = ', round(svc.score(x_test, y_test), 4))

"""
Applying SVM on all the available data, result:

Feature vector length :  8460
28.18  seconds were required for training the SVC
Total number of car samples:  8792
Total number of not_car samples:  8968
Test accuracy for SVC =  0.9811
"""

#Compare results:
n_predict = 15
#print("Values predicted by SVC = ", svc.predict((x_test[0:n_predict])))
#print("Actual labels =           ", y_test[0:n_predict])

"""
1st call:
Values predicted by SVC =  [ 0.  0.  1.  1.  0.  1.  1.  1.  1.  1.  0.  0.  1.  0.  1.]
Actual labels =            [ 0.  0.  1.  1.  0.  1.  1.  1.  1.  1.  0.  0.  1.  0.  1.]

2nd call:
Values predicted by SVC =  [ 0.  0.  1.  1.  0.  1.  1.  0.  1.  1.  1.  1.  0.  0.  1.]
Actual labels =            [ 0.  0.  1.  1.  0.  1.  1.  0.  1.  1.  1.  1.  0.  0.  1.]
"""

model_pickle = {}
#print(type(model_pickle))   #model_pickle is of type dict and hence stores values as key-value pairs

model_pickle['svc'] = svc
model_pickle['scaler'] = X_scaler
model_pickle['orient'] = orient
model_pickle['pix_per_cell'] = pix_per_cell
model_pickle['cell_per_block'] = cell_per_block
model_pickle['spatial_size'] = spatial_size
model_pickle['hist_bins'] = hist_bins
pickle.dump(model_pickle, open("linearSVM training.pkl", "wb"))


## 3. Sliding window

In [ ]:

#implement a sliding window and use your trained classifier to search for vehicles in images
#The function takes an image, start and stop positions in both x and y, window size(x and y dimensions),
#and overlap fraction (for both x and y)
#Default xy_window is 64 X 64 and there is 50% overlap
def slide_window(img, x_start_stop = [None, None], y_start_stop = [None, None],
                 xy_window = (64,64), xy_overlap = (0.5,0.5)):

    """
    print(x_start_stop)
    print(y_start_stop)

    If we call the function plainly without providing the x_start_stop and y_start_stop, then the output is:
    [None, None]
    [None, None]
    """


    #if x and/or y start/stop positions are not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]

    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]

    """
    It will be list as the default args is [None, None]
    print(x_start_stop)
    print(type(x_start_stop))
    
    Output:
    [0, 1280]
    <class 'list'>
    ------------------------
    print(y_start_stop)
    print(type(y_start_stop))
    
    Output:
    [0, 720]
    <class 'list'>
    """

    #Compute the span of the region to be searched
    x_span = x_start_stop[1] - x_start_stop[0]
    y_span = y_start_stop[1] - y_start_stop[0]

    #here we can select how much region of our image we want to scan. As the car can be in any region, and not any sepcific
    #part, we will scan the entire image.
    #print(x_span)  #Output: 1280 and type is int
    #print(y_span)  #Output: 720  and type is int

    #Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0] * (1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1] * (1 - xy_overlap[1]))

    """
    This is the size of the window leaving the overlapped part, eg the four corners
    print(nx_pix_per_step)
    print(type(nx_pix_per_step))

    Output:
    160
    <class 'int'>
    
    print(ny_pix_per_step)
    print(type(ny_pix_per_step))
    
    Output:
    160
    <class 'int'>
    
    """

    #Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0] * xy_overlap[0])
    ny_buffer = np.int(xy_window[1] * xy_overlap[1])
    nx_windows = np.int((x_span - nx_buffer) / nx_pix_per_step)
    # ie 1280 - 40 == 1240. 1240 / 160 == 7
    ny_windows = np.int((y_span - ny_buffer) / ny_pix_per_step)
    #ie 720 - 40 == 680. 680 / 160 == 4
    """ 
    Here, it shows how many pixels are overlapped, ie 200 - 160 = 40 as 20% of 200 is 40
    print(nx_buffer)
    print(ny_buffer)  
    
    Output:
    40
    40

    print(nx_windows)
    print(ny_windows)

    Output:
    7
    4
    """
    #Initialize a list to append window positions to
    window_list = []

    #Loop through finding x and y window positions
    #Note: You could vectorize this step, but in practice you'll be considering windows
    #one by one with your classifier, so looping makes sense

    #ys goes form 0 to 3 ie 4 vertical windows
    #xs goes form 0 to 6 ie 7 horizontal windows
    for ys in range(ny_windows):
        for xs in range(nx_windows):

            #Calculate window position
            startx = xs * nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            # Eg: xs = 0. Therefore, 0 * 160 + 0 = 0
            # Eg: xs = 1. Therefore, 1 * 160 + 0 = 160
            # Eg: xs = 2. Therefore, 2 * 160 + 0 = 320, etc
            #add 200 to the value of startx as our window is 200 X 200
            #Thus draw boxes with x co-ordinates as 0 to 200, 160 to 360, 320 to 520 and so on

            #Sample output of 3 boxes: [((0, 0), (200, 200)), ((160, 0), (360, 200)), ((320, 0), (520, 200)),......
            starty = ys * ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            #add 200 to the value of starty as out window is 200 X 200

            #Append window position to list
            #Add these co-ordinates to the window_list and this list will be used by draw_boxes funtion to draw the boxes
            window_list.append(((startx, starty), (endx, endy)))
    print(window_list)
    #Return the list of windows
    return window_list

image = mpimg.imread('test5.jpg')
#Given image is of the dimensions 1280 X 720

# #Define a function to draw bounding boxes
#It takes as input an image, a list of bounding boxes and an optional color tuple and line thickness
#as inputs and then draws boxes in that color on the output

def draw_boxes(img, bboxes, color = (0,0,255), thick = 6):
    #make a copy of the image and then draw on it
    imcopy = np.copy(img)
    #Iterate through the bounding boxes
    for bbox in bboxes:
        #Draw a rectangle given bbox co-ordinates
        #Write a text above the window to indicate the type of object detected
        #print(bbox[0][0], bbox[1][1]) will print the value of the left top coordinates of the box
        #We print the text 'vehicle' of white color on top of every window and also draw the rectange of blue color.
        font = cv2.FONT_HERSHEY_SIMPLEX
        #cv2.putText(imcopy, 'vehicle', (bbox[0][0], bbox[1][1]), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)

    #return the image copy with the boxes drawn
    return imcopy

w_list = slide_window(image, xy_window=(200,200), xy_overlap=(0.2,0.2))
result = draw_boxes(image, w_list)

#Here we simply display the output
plt.figure(figsize=(20,20))
plt.imshow(result)
plt.title('Image with 200 X 200 window and 20% overlap (normal case)')
plt.show()









